| title               | tags      | created | modified |
|---------------------|-----------|---------|----------|
| dev-rag-deepdoc-pdf | [dev,rag] |  Sun Mar 16 03:09:57 CST 2025 | Sun Mar 16 03:09:57 CST 2025  |


## 需求：
> 基于ragflow进行二次开发，对文档解析（pdf、excel、word）添加图片解析